# CODE THEORY
**Integrant:** Kenny Zhu Ye, Yovany Zhu Ye
- ISIN CODE
- EAN CODE
- ISBN CODE

## SUPPORT FUNCTION

In [2]:
def letter_to_value(l):
    return ord(l.upper()) - 55 if l.isalpha() else int(l)

def generate_code(code):
    numeric_values = []
    for char in code:
        if char.isdigit():
            numeric_values.append(int(char))
        else:   
            numeric_value = letter_to_value(char)
            if numeric_value > 9:
                numeric_values.extend(int(i) for i in str(numeric_value))
            else:
                numeric_values.append(int(numeric_value))
    return numeric_values

## ISIN CODE

In [3]:
def validate_isin(isin_code):

    validation = "invalid ISIN" 
    control_digit = None
    new_isin_code = isin_code

    if len(isin_code) == 12:
        isin_code_process = isin_code[:-1] if isin_code[-1].isalpha() else isin_code
        numeric_values = generate_code(isin_code_process)
        double_numeric_values = [sum(int(digit) for digit in str(numeric_values[i] * 2)) if i % 2 == 0 and numeric_values[i] * 2 > 9
                    else numeric_values[i] * 2 if i % 2 == 0 else numeric_values[i] for i in range(len(numeric_values))]
        
        numeric_value = sum(double_numeric_values)

        if isin_code[-1].isdigit():
            control_digit = isin_code[-1]
            validation = "invalid ISIN" if numeric_value % 10 != 0 else "valid ISIN"
        else:
            control_digit = (10 - numeric_value % 10) % 10
            validation = "valid ISIN" if numeric_value % 10 == 0 else "invalid ISIN"
            new_isin_code = isin_code_process + str(control_digit)
    else:
        numeric_values = numeric_value = double_numeric_values = control_digit = new_isin_code = validation = 'invalid ISIN'

    return numeric_values, double_numeric_values, numeric_value, validation, control_digit, new_isin_code


def found_x_isin(isin_code):
    
    if len(isin_code) == 12:
        x_values = {sum(int(x) for x in str(2*i)): i for i in range(1, 10)}

        if isin_code[2:]:
            has_x = 'X' in isin_code[2:] or 'x' in isin_code[2:]
            x = isin_code.upper().find('X', 2) 

            modified_isin_code = isin_code[:2] + isin_code[2:].replace('X', '0').replace('x', '0')
            numeric_values, double_numeric_values, numeric_value, _, _, _ = validate_isin(modified_isin_code)
            
            if x != -1:
                numeric_values[x+2], double_numeric_values[x+2] = 'X', 'X' if x % 2 == 0 else 'X'
                calculated_digit = (10 - numeric_value % 10) % 10
                digit = x_values[calculated_digit] if x % 2 == 0 else calculated_digit

                complete_isin = list(isin_code)
                complete_isin[x] = str(digit) if has_x else complete_isin.append
                complete_isin = ''.join(complete_isin)
                
            else:
                digit = None
                complete_isin = isin_code 
        else:
            numeric_values, double_numeric_values, numeric_value, digit, complete_isin = 'invalid ISIN', 'invalid ISIN', 'invalid ISIN', 'invalid ISIN', 'invalid ISIN'

    return numeric_values, double_numeric_values, numeric_value, digit, complete_isin

## EAN-13 and EAN-8 CODE

In [4]:
def validate_EAN(code):        

    if len(code) == 13:

        code_process = code[:-1] if code[-1].isalpha() else code
        numeric_values = generate_code(code_process)
        triple_values = [numeric_values[i] * 3 if i % 2 == 1 else numeric_values[i] for i in range(len(numeric_values))]
        numeric_value = sum(triple_values)
        if code[-1].isdigit():
            control_digit = code[-1]
            validation = "invalid EAN13" if numeric_value % 10 != 0 else "valid EAN13"
        else:
            control_digit = (10 - numeric_value % 10) % 10
            validation = "valid EAN13" if numeric_value % 10 == 0 else "invalid EAN13"
            code = code_process + str(control_digit)

    elif len(code) == 8:

        code_process = code[:-1] if code[-1].isalpha() else code
        numeric_values = generate_code(code_process)
        triple_values = [numeric_values[i] * 3 if i % 2 == 0 else numeric_values[i] for i in range(len(numeric_values))]
        numeric_value = sum(triple_values)
        if code[-1].isdigit():
            control_digit = code[-1]
            validation = "invalid EAN8" if numeric_value % 10 != 0 else "valid EAN8"
        else:
            control_digit = (10 - numeric_value % 10) % 10
            validation = "valid EAN8" if numeric_value % 10 == 0 else "invalid EAN8"
            code = code_process + str(control_digit)
    else:
        numeric_values = triple_values = numeric_value = control_digit = validation = code = 'invalid EAN13' if len(code) >= 13 else 'invalid EAN8'

    return numeric_values, triple_values, numeric_value, validation, control_digit, code


def found_x(EAN_code):
    if len(EAN_code) == 13 or len(EAN_code) == 8:
        x = EAN_code.upper().find('X')
        modified_EAN_code = EAN_code.upper().replace('X', '0')
        numeric_values, triple_values, numeric_value, _, _, _ = validate_EAN(modified_EAN_code)

        if x != -1:
            numeric_values[x], triple_values[x] = 'X', 'X'
        
        x_digit = None
        for digit in range(10):
            test_code = EAN_code.upper().replace('X', str(digit))
            _, _, _, validation, _, _ = validate_EAN(test_code)
            if (len(EAN_code) == 13 and validation == "valid EAN13") or (len(EAN_code) == 8 and validation == "valid EAN8"):
                x_digit = digit
                break 

        complete_EAN_code = EAN_code.upper().replace('X', str(x_digit)) if x_digit is not None else 'Invalid code'
    else:
        numeric_values = triple_values = numeric_value = x_digit = complete_EAN_code = 'invalid EAN13' if len(EAN_code) >= 13 else 'invalid EAN8'
    
    return numeric_values, triple_values, numeric_value, x_digit, complete_EAN_code

## ISBN-10 and ISBN-13 CODE

In [5]:
def isbn(isbn_code):

    isbn = isbn_code.replace("-", "")
    
    if len(isbn) == 10:

        numeric_values = [10 if x == 'X' or x == 'x' else int(x) for x in isbn]
        operated_values = [(10-i)*numeric_values[i] for i in range(len(numeric_values))]
        sum_operated_values = sum(operated_values)
        
        if isbn[-1] == 'X' or isbn[-1] == 'x':
            isbn = isbn[:-1] + '10'
        else:
            check_digit = int(isbn[-1])
        
        is_valid = (sum_operated_values + check_digit) % 11 == 0
        validation = "valid ISBN-10" if (sum_operated_values)%11 == 0 else "invalid ISBN-10"
        
    elif len(isbn) == 13:

        numeric_values = [int(x) for x in isbn]
        operated_values = [3*numeric_values[i] if i % 2 else numeric_values[i] for i in range(len(numeric_values))]
        sum_operated_values = sum(operated_values)
        
        check_digit = int(isbn[-1])
        
        is_valid = (sum_operated_values + check_digit) % 10 == 0
        validation = "valid ISBN-13" if is_valid else "invalid ISBN-13"
    
    else:
        numeric_values = operated_values = sum_operated_values = check_digit = validation = 'invalid ISBN-10' if len(isbn) == 10 else 'invalid ISBN-13'
        
    return numeric_values, operated_values , sum_operated_values, validation, check_digit


def isbn10_to_isbn13(isbn10):

    isbn10 = isbn10.replace("-", "")
    if len(isbn10) != 10:
        return None
    
    isbn13_body = '978' + isbn10[:-1]

    check_sum = 0
    for i, digit in enumerate(isbn13_body):
        check_sum += int(digit) if i % 2 == 0 else 3 * int(digit)
    check_digit = 10 - (check_sum % 10)

    if check_digit == 10:
        check_digit = 0  

    isbn13 = isbn13_body + str(check_digit)
    
    return isbn13

## Main Function to ejecute the code

In [6]:
def main_validate(code):
    if ((len(code) >= 13 and code.find('-') == -1) or len(code) == 8) and code[0].isdigit()== True:
        numeric_values, triple_values, numeric_value, validation, control_digit, new_ean_code = validate_EAN(code) 
        print(f"EAN13 Code: {code}")
        print(f"Generated numeric values: {numeric_values}")
        print(f"Operated numeric values: {triple_values}")
        print(f"Sum of operated numeric values: {numeric_value}")
        print(f"Validation: {validation}")
        print(f"Control digit: {control_digit}")
        print(f"New EAN13 Code: {new_ean_code}" if len(code) >= 13 else f"New EAN8 Code: {new_ean_code}")

    if len(code) >= 12 and code[0].isalpha() and code[1].isalpha():
        numeric_values, triple_values, numeric_value, validation, control_digit, new_isin_code = validate_isin(code) 
        print(f"ISIN Code: {code}")
        print(f"Generated numeric values: {numeric_values}")
        print(f"Operated numeric values: {triple_values}")
        print(f"Sum of operated numeric values: {numeric_value}")
        print(f"Validation: {validation}")
        print(f"Control digit: {control_digit}")
        print(f"New ISIN Code: {new_isin_code}")  

    if len(code) == 10 or (len(code) >= 13 and code.find('-') != -1):
        numeric_values, operated_values, sum_operated_values, validation, check_digit = isbn(code)
        print(f"ISBN Code: {code}")
        print(f"Numeric values: {numeric_values}")
        print(f"Operated values: {operated_values}")
        print(f"Sum of operated values: {sum_operated_values}")
        print(f"Validation: {validation}")
        print(f"Check digit: {check_digit}")
    return

def main_x(code):
    if len(code) >= 13 or len(code) == 8:
        numeric_values, triple_values, numeric_value, x_digit, complete_EAN13 = found_x(code)
        print(f"EAN13 Code: {code}")
        print(f"Generated numeric values: {numeric_values}")
        print(f"Operated numeric values: {triple_values}")
        print(f"Sum of operated numeric values: {numeric_value}")
        print(f"The digit to replace X: {x_digit}")
        print(f"New generated EAN13: {complete_EAN13}" if len(code) >= 13 else f"New generated EAN8: {complete_EAN13}")

    if len(code) == 12:
        numeric_values, double_numeric_values, numeric_value, digit, complete_isin = found_x_isin(code)
        print(f"ISIN Code: {code}")
        print(f"Generated numeric values: {numeric_values}")
        print(f"Operated numeric values: {double_numeric_values}")
        print(f"Sum of operated numeric values: {numeric_value}")
        print(f"The digit to replace X: {digit}")
        print(f"New generated ISIN: {complete_isin}")  
    return

## Unit Test

### validate ISIN CODE

In [7]:
isin_example = "US7960502018"
main_validate(isin_example)

ISIN Code: US7960502018
Generated numeric values: [3, 0, 2, 8, 7, 9, 6, 0, 5, 0, 2, 0, 1, 8]
Operated numeric values: [6, 0, 4, 8, 5, 9, 3, 0, 1, 0, 4, 0, 2, 8]
Sum of operated numeric values: 50
Validation: valid ISIN
Control digit: 8
New ISIN Code: US7960502018


### control digit ISIN CODE

In [8]:
isin_example = "NL001158514X"
main_validate(isin_example)

ISIN Code: NL001158514X
Generated numeric values: [2, 3, 2, 1, 0, 0, 1, 1, 5, 8, 5, 1, 4]
Operated numeric values: [4, 3, 4, 1, 0, 0, 2, 1, 1, 8, 1, 1, 8]
Sum of operated numeric values: 34
Validation: invalid ISIN
Control digit: 6
New ISIN Code: NL0011585146


### found X in ISIN CODE

In [9]:
isin_example = "MX20273749X7"
main_x(isin_example)

ISIN Code: MX20273749X7
Generated numeric values: [2, 2, 3, 3, 2, 0, 2, 7, 3, 7, 4, 9, 'X', 7]
Operated numeric values: [4, 2, 6, 3, 4, 0, 4, 7, 6, 7, 8, 9, 'X', 7]
Sum of operated numeric values: 67
The digit to replace X: 6
New generated ISIN: MX2027374967


### validate EAN-13 code

In [10]:
EAN13_example = "4902778913970"
main_validate(EAN13_example)

EAN13 Code: 4902778913970
Generated numeric values: [4, 9, 0, 2, 7, 7, 8, 9, 1, 3, 9, 7, 0]
Operated numeric values: [4, 27, 0, 6, 7, 21, 8, 27, 1, 9, 9, 21, 0]
Sum of operated numeric values: 140
Validation: valid EAN13
Control digit: 0
New EAN13 Code: 4902778913970


### Control digit EAN-13 code

In [11]:
EAN13_example1 = "780389328012X"
main_validate(EAN13_example1)   

EAN13 Code: 780389328012X
Generated numeric values: [7, 8, 0, 3, 8, 9, 3, 2, 8, 0, 1, 2]
Operated numeric values: [7, 24, 0, 9, 8, 27, 3, 6, 8, 0, 1, 6]
Sum of operated numeric values: 99
Validation: invalid EAN13
Control digit: 1
New EAN13 Code: 7803893280121


In [12]:
EAN13_example2 = "7590167890234X"
main_validate(EAN13_example2)

EAN13 Code: 7590167890234X
Generated numeric values: invalid EAN13
Operated numeric values: invalid EAN13
Sum of operated numeric values: invalid EAN13
Validation: invalid EAN13
Control digit: invalid EAN13
New EAN13 Code: invalid EAN13


### found X EAN-13 code

In [13]:
EAN13_example3 = '843x554161789'
main_x(EAN13_example3)

EAN13 Code: 843x554161789
Generated numeric values: [8, 4, 3, 'X', 5, 5, 4, 1, 6, 1, 7, 8, 9]
Operated numeric values: [8, 12, 3, 'X', 5, 15, 4, 3, 6, 3, 7, 24, 9]
Sum of operated numeric values: 99
The digit to replace X: 7
New generated EAN13: 8437554161789


In [14]:
EAN13_example4 = '5725351455x67'
main_x(EAN13_example4)

EAN13 Code: 5725351455x67
Generated numeric values: [5, 7, 2, 5, 3, 5, 1, 4, 5, 5, 'X', 6, 7]
Operated numeric values: [5, 21, 2, 15, 3, 15, 1, 12, 5, 15, 'X', 18, 7]
Sum of operated numeric values: 119
The digit to replace X: 1
New generated EAN13: 5725351455167


### validate EAN-8 code

In [15]:
ean8_example = "77000260"
main_validate(ean8_example)

EAN13 Code: 77000260
Generated numeric values: [7, 7, 0, 0, 0, 2, 6, 0]
Operated numeric values: [21, 7, 0, 0, 0, 2, 18, 0]
Sum of operated numeric values: 48
Validation: invalid EAN8
Control digit: 0
New EAN8 Code: 77000260


In [16]:
ean8_example1 = "12344567"
main_validate(ean8_example1)

EAN13 Code: 12344567
Generated numeric values: [1, 2, 3, 4, 4, 5, 6, 7]
Operated numeric values: [3, 2, 9, 4, 12, 5, 18, 7]
Sum of operated numeric values: 60
Validation: valid EAN8
Control digit: 7
New EAN8 Code: 12344567


### found X EAN-8 code

In [17]:
ean8_example = "735135X3"
main_x(ean8_example)

EAN13 Code: 735135X3
Generated numeric values: [7, 3, 5, 1, 3, 5, 'X', 3]
Operated numeric values: [21, 3, 15, 1, 9, 5, 'X', 3]
Sum of operated numeric values: 57
The digit to replace X: 1
New generated EAN8: 73513513


### validate ISBN-10 code 

In [18]:
isbn10_example = "0-534-91500-0"
main_validate(isbn10_example)

ISBN Code: 0-534-91500-0
Numeric values: [0, 5, 3, 4, 9, 1, 5, 0, 0, 0]
Operated values: [0, 45, 24, 28, 54, 5, 20, 0, 0, 0]
Sum of operated values: 176
Validation: valid ISBN-10
Check digit: 0


In [19]:
isbn10_example1 = "0-534-91700-0"
main_validate(isbn10_example1)

ISBN Code: 0-534-91700-0
Numeric values: [0, 5, 3, 4, 9, 1, 7, 0, 0, 0]
Operated values: [0, 45, 24, 28, 54, 5, 28, 0, 0, 0]
Sum of operated values: 184
Validation: invalid ISBN-10
Check digit: 0


### validate ISBN-13 code   

In [20]:
isbn13_example = "978-0-306-40615-7"
main_validate(isbn13_example)

ISBN Code: 978-0-306-40615-7
Numeric values: [9, 7, 8, 0, 3, 0, 6, 4, 0, 6, 1, 5, 7]
Operated values: [9, 21, 8, 0, 3, 0, 6, 12, 0, 18, 1, 15, 7]
Sum of operated values: 100
Validation: invalid ISBN-13
Check digit: 7


### convert ISBN-10 to ISBN-13

In [21]:
isbn10_sample = "9588252008"
isbn13_converted = isbn10_to_isbn13(isbn10_sample)

print(f"ISBN-10: {isbn10_sample}")
print(f"The ISBN-13 converted: {isbn13_converted}")

ISBN-10: 9588252008
The ISBN-13 converted: 9789588252001


## Activity Test

### Punto 1, ISBN-10 validate

In [22]:
isbn10_a = '0-7645-2641-3'
isbn10_b = '4-10-206602-7'
isbn10_c = '9-6521-7431-8'
isbn10_d = '8-4983-8149-5'

numeric_valuesa, operated_valuea, sum_operated_valuesa, validationa, chek_digita = isbn(isbn10_a)
numeric_valuesb, operated_valueb, sum_operated_valuesb, validationb, chek_digitb = isbn(isbn10_b)
numeric_valuesc, operated_valuec, sum_operated_valuesc, validationc, chek_digitc = isbn(isbn10_c)
numeric_valuesd, operated_valued, sum_operated_valuesd, validationd, chek_digitd = isbn(isbn10_d)

for i in [isbn10_a, isbn10_b, isbn10_c, isbn10_d]:
    numeric_values, operated_values, sum_operated_values, validation, check_digit = isbn(i)
    print(f"ISBN-10: {i}")
    print(f"Numeric values: {numeric_values}")
    print(f"Operated values: {operated_values}")
    print(f"Sum of operated values: {sum_operated_values}")
    print(f"Validation: {validation}")
    print(f"\n")

ISBN-10: 0-7645-2641-3
Numeric values: [0, 7, 6, 4, 5, 2, 6, 4, 1, 3]
Operated values: [0, 63, 48, 28, 30, 10, 24, 12, 2, 3]
Sum of operated values: 220
Validation: valid ISBN-10


ISBN-10: 4-10-206602-7
Numeric values: [4, 1, 0, 2, 0, 6, 6, 0, 2, 7]
Operated values: [40, 9, 0, 14, 0, 30, 24, 0, 4, 7]
Sum of operated values: 128
Validation: invalid ISBN-10


ISBN-10: 9-6521-7431-8
Numeric values: [9, 6, 5, 2, 1, 7, 4, 3, 1, 8]
Operated values: [90, 54, 40, 14, 6, 35, 16, 9, 2, 8]
Sum of operated values: 274
Validation: invalid ISBN-10


ISBN-10: 8-4983-8149-5
Numeric values: [8, 4, 9, 8, 3, 8, 1, 4, 9, 5]
Operated values: [80, 36, 72, 56, 18, 40, 4, 12, 18, 5]
Sum of operated values: 341
Validation: valid ISBN-10




### Punto 2 EAN-13 validate

In [23]:
ean13_a = '7591147307013'
ean13_b = '5814709192389'
ean13_c = '4967834241223'
ean13_d = '4897005412190'

numeric_valuesa, triple_valuesa, numeric_valuea, validationa, control_digita, new_ean_codea = validate_EAN(ean13_a)

for i in [ean13_a, ean13_b, ean13_c, ean13_d]:
    numeric_values, triple_values, numeric_value, validation, control_digit, new_ean_code = validate_EAN(i)
    print(f"EAN13 Code: {i}")
    print(f"Generated numeric values: {numeric_values}")
    print(f"Operated numeric values: {triple_values}")
    print(f"Sum of operated numeric values: {numeric_value}")
    print(f"Validation: {validation}")
    print(f"\n")

EAN13 Code: 7591147307013
Generated numeric values: [7, 5, 9, 1, 1, 4, 7, 3, 0, 7, 0, 1, 3]
Operated numeric values: [7, 15, 9, 3, 1, 12, 7, 9, 0, 21, 0, 3, 3]
Sum of operated numeric values: 90
Validation: valid EAN13


EAN13 Code: 5814709192389
Generated numeric values: [5, 8, 1, 4, 7, 0, 9, 1, 9, 2, 3, 8, 9]
Operated numeric values: [5, 24, 1, 12, 7, 0, 9, 3, 9, 6, 3, 24, 9]
Sum of operated numeric values: 112
Validation: invalid EAN13


EAN13 Code: 4967834241223
Generated numeric values: [4, 9, 6, 7, 8, 3, 4, 2, 4, 1, 2, 2, 3]
Operated numeric values: [4, 27, 6, 21, 8, 9, 4, 6, 4, 3, 2, 6, 3]
Sum of operated numeric values: 103
Validation: invalid EAN13


EAN13 Code: 4897005412190
Generated numeric values: [4, 8, 9, 7, 0, 0, 5, 4, 1, 2, 1, 9, 0]
Operated numeric values: [4, 24, 9, 21, 0, 0, 5, 12, 1, 6, 1, 27, 0]
Sum of operated numeric values: 110
Validation: valid EAN13




### Punto 3 ISIN validate

In [24]:
isbn10_a = 'US7740502024'
isbn10_b = 'NA0203115054'
isbn10_c = 'US8857934101'
isbn10_d = 'US5949181045'

numeric_valuesa, operated_valuea, sum_operated_valuesa, validationa, chek_digita, new_isin_code = validate_isin(isbn10_a)

for i in [isbn10_a, isbn10_b, isbn10_c, isbn10_d]:
    numeric_values, operated_values, sum_operated_values, validation, check_digit, new_isin_code = validate_isin(i)
    print(f"ISIN Code: {i}")
    print(f"Generated numeric values: {numeric_values}")
    print(f"Operated numeric values: {operated_values}")
    print(f"Sum of operated numeric values: {sum_operated_values}")
    print(f"Validation: {validation}")
    print(f"\n")

ISIN Code: US7740502024
Generated numeric values: [3, 0, 2, 8, 7, 7, 4, 0, 5, 0, 2, 0, 2, 4]
Operated numeric values: [6, 0, 4, 8, 5, 7, 8, 0, 1, 0, 4, 0, 4, 4]
Sum of operated numeric values: 51
Validation: invalid ISIN


ISIN Code: NA0203115054
Generated numeric values: [2, 3, 1, 0, 0, 2, 0, 3, 1, 1, 5, 0, 5, 4]
Operated numeric values: [4, 3, 2, 0, 0, 2, 0, 3, 2, 1, 1, 0, 1, 4]
Sum of operated numeric values: 23
Validation: invalid ISIN


ISIN Code: US8857934101
Generated numeric values: [3, 0, 2, 8, 8, 8, 5, 7, 9, 3, 4, 1, 0, 1]
Operated numeric values: [6, 0, 4, 8, 7, 8, 1, 7, 9, 3, 8, 1, 0, 1]
Sum of operated numeric values: 63
Validation: invalid ISIN


ISIN Code: US5949181045
Generated numeric values: [3, 0, 2, 8, 5, 9, 4, 9, 1, 8, 1, 0, 4, 5]
Operated numeric values: [6, 0, 4, 8, 1, 9, 8, 9, 2, 8, 2, 0, 8, 5]
Sum of operated numeric values: 70
Validation: valid ISIN




## punto 4, hallar cifra de control

In [25]:
isbn13_digit = '978410206602X'
isbn10_digit = '84-567-9124-X'
ean13_digit = '885909650132X'
ean8_digit = '4012276X'

numeric_valuesisbn13, triple_valuesisbn13, numeric_valueisbn13, validationisbn13, checkdigitisbn13, new_ean_codeisbn13 = validate_EAN(isbn13_digit)
numeric_valuesisbn10, operated_valuesisbn10, sum_operated_valuesisbn10, validationisbn10, checkdigitisbn10 = isbn(isbn10_digit)
numeric_valuesean13, triple_valuesean13, numeric_valueean13, validationean13, checkdigitean13, new_ean_code13 = validate_EAN(ean13_digit)
numeric_valuesean8, triple_valuesean8, numeric_valueean8, validationean8, checkdigitean8, new_ean_code8 = validate_EAN(ean8_digit)  

UnboundLocalError: cannot access local variable 'check_digit' where it is not associated with a value

In [ ]:
print(f"ISBN-13 Code: {isbn13_digit}")
print(f"Generated numeric values: {numeric_valuesisbn13}")  
print(f"Operated numeric values: {triple_valuesisbn13}")
print(f"Sum of operated numeric values: {numeric_valueisbn13}")
print(f"check digit: {checkdigitisbn13}")
print(f"New ISBN13 Code: {new_ean_codeisbn13}")

ISBN-13 Code: 978410206602X
Generated numeric values: [9, 7, 8, 4, 1, 0, 2, 0, 6, 6, 0, 2]
Operated numeric values: [9, 21, 8, 12, 1, 0, 2, 0, 6, 18, 0, 6]
Sum of operated numeric values: 83
check digit: 7
New ISBN13 Code: 9784102066027


In [ ]:
print(f"ean13 Code: {ean13_digit}")
print(f"Generated numeric values: {numeric_valuesean13}")
print(f"Operated numeric values: {triple_valuesean13}") 
print(f"Sum of operated numeric values: {numeric_valueean13}")
print(f"check digit: {checkdigitean13}")
print(f"New ean13 Code: {new_ean_code13}")

ean13 Code: 885909650132X
Generated numeric values: [8, 8, 5, 9, 0, 9, 6, 5, 0, 1, 3, 2]
Operated numeric values: [8, 24, 5, 27, 0, 27, 6, 15, 0, 3, 3, 6]
Sum of operated numeric values: 124
check digit: 6
New ean13 Code: 8859096501326


In [ ]:
print(f"ean8 Code: {ean8_digit}")   
print(f"Generated numeric values: {numeric_valuesean8}")
print(f"Operated numeric values: {triple_valuesean8}")
print(f"Sum of operated numeric values: {numeric_valueean8}")
print(f"check digit: {checkdigitean8}")
print(f"New ean8 Code: {new_ean_code8}")

ean8 Code: 4012276X
Generated numeric values: [4, 0, 1, 2, 2, 7, 6]
Operated numeric values: [12, 0, 3, 2, 6, 7, 18]
Sum of operated numeric values: 48
check digit: 2
New ean8 Code: 40122762
